In [1]:
import os

import numpy as np
import pandas as pd

from flow.controllers.base_controller import BaseController
from flow.controllers.routing_controllers import ContinuousRouter
from flow.core.experiment import Experiment
from flow.core.params import EnvParams
from flow.core.params import InitialConfig
from flow.core.params import NetParams
from flow.core.params import SumoParams
from flow.core.params import VehicleParams
from flow.core.params import SumoCarFollowingParams
from flow.envs.ring.accel import AccelEnv
from flow.networks.ring import RingNetwork

In [2]:
class NaiveBaselineController(BaseController):
    def __init__(self, veh_id, v0=30, s0=20, car_following_params=None):
        """
        veh_id: unique vehicle identifier
        v0: desirable velocity, in m/s
        s0: headway threshold before braking, in m
        """
        
        BaseController.__init__(self, veh_id, car_following_params)
        self.v0 = v0
        self.s0 = s0

    def get_accel(self, env):
        v = env.k.vehicle.get_speed(self.veh_id)
        lead_id = env.k.vehicle.get_leader(self.veh_id)
        h = env.k.vehicle.get_headway(self.veh_id)

        if h < 10:  # car in front less than 10m away
            return -2
        elif v < self.v0:
            return 2
        else:
            return 0

In [3]:
exp_name = "ring_network_baseline"

num_vehicles = 20

vehicles = VehicleParams()
vehicles.add(
    "robot",
    acceleration_controller=(NaiveBaselineController, {}),
    routing_controller=(ContinuousRouter, {}),
    num_vehicles=num_vehicles,
)

net_params = NetParams(additional_params={
    'length': 200*np.pi,  # 200m diameter
    'lanes': 1, 
    'speed_limit': 30, 
    'resolution': 40,
})

# initial_config = InitialConfig(spacing="uniform", perturbation=3)
initial_config = InitialConfig(spacing="random", min_gap=2)

env_params = EnvParams(
    additional_params={
        'max_accel': 2, 
        'max_decel': 2, 
        'target_velocity': 30, 
        'sort_vehicles': False,
    },
    horizon=1000,  # number of time steps
)

sim_params = SumoParams(sim_step=0.1, render=True, emission_path='data')

In [4]:
flow_params = {
    'exp_tag': exp_name,
    'env_name': AccelEnv,
    'network': RingNetwork,
    'simulator': 'traci',
    'sim': sim_params,
    'env': env_params,
    'net': net_params,
    'veh': vehicles,
    'initial': initial_config,
}

exp = Experiment(flow_params)

# run the sumo simulation
_ = exp.run(1, convert_to_csv=True)

Round 0, return: 258.39632963279075
data/ring_network_baseline_20210413-1827071618334827.208069-0_emission.csv data
Average, std returns: 258.39632963279075, 0.0
Average, std velocities: 9.823673629442021, 0.0
Average, std outflows: 0.0, 0.0
Total time: 12.888464450836182
steps/second: 103.95298336654702


In [5]:
# view results

emission_location = os.path.join(exp.env.sim_params.emission_path, exp.env.network.name)
print(emission_location)

data/ring_network_baseline_20210413-1827071618334827.208069


In [6]:
data = pd.read_csv(emission_location + '-0_emission.csv')
data.head()

,time,id,x,y,speed,headway,leader_id,target_accel_with_noise_with_failsafe,target_accel_no_noise_no_failsafe,target_accel_with_noise_no_failsafe,target_accel_no_noise_with_failsafe,realized_accel,road_grade,edge_id,lane_number,distance,relative_position,follower_id,leader_rel_speed
0,0.0,robot_0,117.557337,-0.058550,0.000000,59.482138,robot_1,NaN,NaN,NaN,NaN,0.000000,0,bottom,0,0.000000,17.347827,robot_8,0.0
1,0.1,robot_0,117.577115,-0.054909,0.198020,59.482138,robot_1,2.0,2.0,2.0,2.0,1.980198,0,bottom,0,0.019802,17.367629,robot_8,0.0
2,0.2,robot_0,117.616671,-0.047628,0.396040,59.482138,robot_1,2.0,2.0,2.0,2.0,1.980198,0,bottom,0,0.059406,17.407233,robot_8,0.0
3,0.3,robot_0,117.676004,-0.036706,0.594059,59.482138,robot_1,2.0,2.0,2.0,2.0,1.980198,0,bottom,0,0.118812,17.466639,robot_8,0.0
4,0.4,robot_0,117.755116,-0.022143,0.792079,59.482138,robot_1,2.0,2.0,2.0,2.0,1.980198,0,bottom,0,0.198020,17.545847,robot_8,0.0
